In [40]:
# импорт необходимых библиотек

import pandas as pd
from pandas import json_normalize

from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import json

import requests

In [41]:
link = 'https://cbbh.ba/CurrencyExchange/'  # ссылка на сайт ЦБ Боснии и Герцоговины, откуда будем брать данные по курсу

In [42]:
options = Options()
options.headless = True  # чтобы при запуске кода не открывался отдельное окно браузера 

In [43]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)  # запуск вебдрайвера
driver.get(link)  # передаем ссылку

C:\Users\huawei\AppData\Local\Temp\ipykernel_8080\1447797879.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)  # запуск вебдрайвера


In [44]:
driver.find_element(By.ID, 'dateFrom').clear()
driver.find_element(By.ID, 'dateTo').clear()  # очистили окошки для ввода дат от текущей даты

In [45]:
start_date = input('Введите дату начала в формате дд.мм.гггг')
finish_date = input('Введите дату конца в формате дд.мм.гггг')  # просим пользователя ввести даты начала и конца периода

Введите дату начала в формате дд.мм.гггг12.10.2022
Введите дату конца в формате дд.мм.гггг22.10.2022


In [46]:
date1 = driver.find_element(By.ID, 'dateFrom')
date2 = driver.find_element(By.ID, 'dateTo')  # вебдрайвер ищет окошки для ввода переданных пользователем дат

In [47]:
date1.send_keys(start_date)
date2.send_keys(finish_date)  # ввод переданных дат

In [48]:
driver.find_element(By.XPATH, "/html/body/section/div/div[1]/div/div[2]/div[3]/a[1]").click()  #нажимаем на кнопку,
                                                                                               #чтобы получить json файл

In [49]:
url = driver.current_url  # берем ссылку на json

In [50]:
response = requests.get(url)  # отправляем запрос, чтобы взять данные 

In [52]:
data = response.json()  # сохраняем json в отдельную переменную

In [53]:
df = pd.DataFrame(columns=('Дата', 'Курс'))  # делаем заготовку под будущий датафрейм

In [54]:
# нам из всех полученных данных нужна только дата и курс к доллару, поэтому сохраняем эти значения в списки
dates = []
currencies = []
for i in range(len(data)):
    dates.append(data[i]['Date'])
    currencies.append(data[i]['CurrencyExchangeItems'][13]['Middle'])

In [55]:
# передаем сохраненные значения в столбцы датафрейма
df['Дата'] = dates
df['Курс'] = currencies

In [56]:
# приводим все к правильному типу данных
df['Дата'] = pd.to_datetime(df['Дата'])
df['Курс'] = df['Курс'].str.replace(',', '.')
df['Курс'] = df['Курс'].astype(float)

In [57]:
# приводим дату к формату дд/мм/гггг
df['Дата'] = df['Дата'].dt.strftime('%d/%m/%Y')

In [58]:
# сохраняем в excel
df.to_excel('cbbh.xlsx', index=False)